In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, HubertModel
import pandas
import soundfile as sf
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "code" / "experiments" / "data" / "ASVSpoof2019"



In [4]:
class EnsembleModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        self.hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960")
        combined_dim = self.wav2vec2.config.hidden_size + self.hubert.config.hidden_size

        self.classifier = nn.Linear(combined_dim, num_classes)
    
    def forward(self, x):
        w2v_out = self.wav2vec2(x).last_hidden_state.mean(dim=1)
        hubert_out = self.hubert(x).last_hidden_state.mean(dim=1)

        combined = torch.cat((w2v_out, hubert_out), dim=1)

        return self.classifier(combined)
    
class ASVSpoofDataset(Dataset):
    
    def __init__(self, flac_dir, labels_path):
        """
        Returns all the directory where the flac_files are located, returns the files itself,
        returns the dataset with filenames, targets, speaker ID, and type attack ID.
        Also returns the list of filenames, and target dictionary
        """
        self.flac_dir = flac_dir
        self.files = sorted(Path(flac_dir).glob("*.flac"))
        self.labels_df = pandas.read_csv(labels_path, sep=r"\s+", header=None)
        self.file_names = self.labels_df[1]
        self.target = dict(zip(self.labels_df[1], self.labels_df[4]))

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self,idx):
        file_path = self.files[idx]
  
        audio, sr = sf.read(str(file_path), dtype="float32", always_2d=True)
        x = torch.from_numpy(audio.T)
        x = x.mean(dim=0)     
 
        x = self.normalize_duration(x)
        file_name = file_path.stem
        target_str = self.target.get(file_name)

        y = 1 if target_str == 'bonafide' else 0
        return x, torch.tensor(y).long()

    def normalize_duration(self, x):
        """
        x: torch.Tensor формы (samples,) или (1, samples)
        """
        TARGET_SEC = 4.0
        TARGET_LEN = int(16000 * TARGET_SEC)
        
        if x.ndim > 1:
            x = x.squeeze()

        cur_len = x.shape[0]

        if cur_len > TARGET_LEN:
            start = torch.randint(0, cur_len - TARGET_LEN + 1, (1,)).item()
            return x[start : start + TARGET_LEN]
        
        elif cur_len < TARGET_LEN:
            pad_len = TARGET_LEN - cur_len
            return torch.nn.functional.pad(x, (0, pad_len), mode='constant', value=0)
        
        return x.float()

In [5]:
project_root = Path(r"C:\Users\egorv\Desktop\BProj")
   
model_path = project_root / "model_20260129_011815_3.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EnsembleModel().to(device)
state = torch.load(model_path, map_location=device)
model.load_state_dict(state)

Loading weights: 100%|██████████| 211/211 [00:00<00:00, 1200.66it/s, Materializing param=masked_spec_embed]                                            
Wav2Vec2Model LOAD REPORT from: facebook/wav2vec2-base
Key                          | Status     |  | 
-----------------------------+------------+--+-
quantizer.weight_proj.bias   | UNEXPECTED |  | 
quantizer.weight_proj.weight | UNEXPECTED |  | 
project_q.bias               | UNEXPECTED |  | 
project_hid.weight           | UNEXPECTED |  | 
quantizer.codevectors        | UNEXPECTED |  | 
project_q.weight             | UNEXPECTED |  | 
project_hid.bias             | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 211/211 [00:00<00:00, 903.97it/s, Materializing param=masked_spec_embed]                                            


<All keys matched successfully>

In [ ]:
test_flac_dir = (DATA_DIR/ "LA" / "ASVSpoof2019_LA_eval" / "flac")
test_file = DATA_DIR / "LA" / "ASVspoof2019_LA_cm_protocols" / "ASVspoof2019.LA.cm.eval.trl.txt"

test_dataset = ASVSpoofDataset(test_flac_dir, test_file)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=0, shuffle=False)


def collect_scores(model, loader, device):
    model.eval()
    y_true = []
    y_score = []

    with torch.no_grad():
        for audios, labels in loader:
            audios = audios.to(device)
            labels = labels.to(device)

            logits = model(audios)
            probs = F.softmax(logits, dim=1)
            score_bonafide = probs[:, 1]

            y_true.extend(labels.cpu().numpy().tolist())
            y_score.extend(score_bonafide.cpu().numpy().tolist())

    return np.array(y_true), np.array(y_score)

from sklearn.metrics import roc_curve

def compute_eer_sklearn(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    fnr = 1 - tpr

    idx = np.nanargmin(np.abs(fpr - fnr))
    eer = (fpr[idx] + fnr[idx]) / 2.0
    thr = thresholds[idx]
    return eer, thr

y_true, y_score = collect_scores(model, test_loader, device)
eer, thr = compute_eer_sklearn(y_true, y_score)

print("EER:", eer)
print("Best threshold:", thr)
